In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from functools import lru_cache
import re

In [72]:
df_statements = pd.read_csv("data/annual_statements.csv")
df_prices = pd.read_csv("data/monthly_returns.csv")

In [73]:
df_statements['datadate'] = pd.to_datetime(df_statements['datadate'])

In [4]:
df_statements.head()

,Unnamed: 0,gvkey,tic,fyear,datadate,conm,revt,ni,emp,prcc_f,ceq,oancf,at,dltt,act,lct,csho,cogs
0,0,1004,AIR,2002.0,2003-05-31,AAR CORP,606.337,-12.410,2.1,4.50,294.988,34.733,686.621,164.658,396.412,203.575,31.851,496.747
1,1,1004,AIR,2003.0,2004-05-31,AAR CORP,651.958,3.504,2.3,9.58,301.684,14.572,709.292,248.666,432.204,131.261,32.245,523.302
2,2,1004,AIR,2004.0,2005-05-31,AAR CORP,747.848,15.453,2.6,16.04,314.744,50.938,732.230,227.159,474.542,160.025,32.586,598.172
3,3,1004,AIR,2005.0,2006-05-31,AAR CORP,897.284,35.163,3.3,24.08,422.717,-40.482,978.819,318.576,624.454,187.788,36.654,704.081
4,4,1004,AIR,2006.0,2007-05-31,AAR CORP,1061.169,58.660,3.9,32.50,494.243,-21.239,1067.633,253.611,645.721,256.506,37.729,837.171


In [17]:
df_prices.head()

,Unnamed: 0,AIR,ADCT.1,AAL,ASA,PNW,PRG,ABT,SERV.1,WDDD,...,APG,NVT,CSTPF,ACA,CTRM,BBUS,IMUX,ARMP,SNOW,HYFM
0,2003-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-02,-0.065126,NaN,NaN,-0.094070,-0.006187,-0.085308,-0.060030,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
2,2003-03,-0.150562,NaN,NaN,-0.037873,0.088408,0.044042,0.055868,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586207,NaN,NaN
3,2003-04,0.026455,NaN,NaN,0.015850,-0.000601,0.049628,0.080297,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586957,NaN,NaN
4,2003-05,0.159794,NaN,NaN,0.018440,0.154016,0.026004,0.103219,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.643836,NaN,NaN


In [60]:
def compute_weighted_sales_growth(df_statements: pd.DataFrame, min_obs=3) -> pd.DataFrame:
    """
    Lakonishok–Shleifer–Vishny (1994) GS: recency-weighted (5..1) average of
    within-year ranks of past 5Y sales growth (Y-1..Y-5), per formation year.
    Returns: columns ['gvkey','tic','gs','formation_year'].
    Firms must have >= min_obs valid past-year growth observations.
    """
    df = df_statements.copy()

    # Ensure needed cols and types
    for col in ['gvkey','tic','fyear','revt']:
        if col not in df.columns:
            raise ValueError(f"Missing column '{col}'")
    df = df.sort_values(['gvkey','fyear']).copy()
    df['fyear'] = pd.to_numeric(df['fyear'], errors='coerce').astype('Int64')

    # Sales growth per firm-year
    df['revt_lag'] = df.groupby('gvkey', group_keys=False)['revt'].shift(1)
    df['sales_growth'] = (df['revt'] / df['revt_lag']) - 1.0

    # Formation year = fyear + 1
    df['formation_year'] = df['fyear'] + 1

    rows = []

    for Y in np.sort(df['formation_year'].dropna().unique().astype(int)):
        # Past five fiscal years: Y-1..Y-5
        window_years = [Y - k for k in range(1, 6)]
        g = df.loc[df['fyear'].isin(window_years), ['gvkey','tic','fyear','sales_growth']].dropna()

        if g.empty:
            continue

        # Rank within each fiscal year; higher growth => higher rank
        g = g.copy()
        g['rank'] = g.groupby('fyear', group_keys=False)['sales_growth'].rank(method='average')
        # Normalize ranks to 0..1 per year to handle different cross-section sizes
        max_rank = g.groupby('fyear', group_keys=False)['rank'].transform('max')
        g['rank_pct'] = np.where(max_rank > 0, g['rank'] / max_rank, np.nan)

        # Recency weights: Y-1->5, Y-2->4, ..., Y-5->1
        w = 6 - (Y - g['fyear'])
        g['weight'] = w.astype(float)
        g = g.loc[g['weight'].between(1, 5)]

        if g.empty:
            continue

        # Keep firms with at least min_obs valid past-year growths
        g_valid = (g.groupby(['gvkey','tic'], group_keys=False)
                     .filter(lambda grp: grp['sales_growth'].notnull().sum() >= min_obs))

        if g_valid.empty:
            continue

        # Weighted average of rank_pct using 'weight' per firm
        # Return a 1-row Series so .reset_index() works reliably
        gs = (g_valid.groupby(['gvkey','tic'])
                      .apply(lambda x: pd.Series({
                          'gs': np.average(x['rank_pct'].to_numpy(),
                                           weights=x['weight'].to_numpy())
                      }))
                      .reset_index())

        gs['formation_year'] = Y
        rows.append(gs)

    if not rows:
        # No valid GS for any formation year
        return pd.DataFrame(columns=['gvkey','tic','gs','formation_year'])

    return pd.concat(rows, ignore_index=True)

gs_table = compute_weighted_sales_growth(df_statements)
print(gs_table.head())

/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({
/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrame

   gvkey     tic        gs  formation_year
0   1004     AIR  0.575769            2006
1   1072     AVX  0.384977            2006
2   1082  SERV.1  0.054370            2006
3   1111  ATVI.1  0.547543            2006
4   1173   AIM.1  0.478552            2006


/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93162/389755030.py:59: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [61]:
def build_decile_memberships_with_gs(df_statements: pd.DataFrame,
                                     gs_table: pd.DataFrame,
                                     id_col: str = 'tic',
                                     n_bins: int = 10):
    """
    Return nested dict: deciles_by_year[char_key][year][decile] -> list of tickers.
    Uses precomputed gs_table with columns ['gvkey','tic','gs','formation_year'].
    """

    df = df_statements.copy()

    # --- Basic sanity ---
    req = {'gvkey','tic','fyear','datadate','ni','oancf','ceq','prcc_f','csho'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"df_statements missing columns: {missing}")

    # --- Clean & sort ---
    df['datadate'] = pd.to_datetime(df['datadate'])
    df = df.sort_values(['gvkey','datadate']).reset_index(drop=True)

    # --- Market equity & core ratios (at fiscal-year end) ---
    df['ME'] = df['prcc_f'] * df['csho']     # assumes shares in millions, price in $
    df.loc[~(df['ME'] > 0), 'ME'] = np.nan

    df['bm'] = df['ceq']   / df['ME']        # Book-to-Market
    df['ep'] = df['ni']    / df['ME']        # Earnings-to-Price
    df['cp'] = df['oancf'] / df['ME']        # CashFlow-to-Price

    # --- Formation year (end of April Y uses fyear=Y-1 financials) ---
    df['formation_year'] = pd.to_numeric(df['fyear'], errors='coerce').astype('Int64') + 1

    # --- Merge precomputed GS ---
    # Ensure types align; drop dup rows in gs_table if any
    gs = gs_table.copy()
    if 'formation_year' in gs.columns:
        gs['formation_year'] = pd.to_numeric(gs['formation_year'], errors='coerce').astype('Int64')
    gs = (gs
          .dropna(subset=['gvkey','tic','formation_year'])
          .drop_duplicates(subset=['gvkey','tic','formation_year'], keep='last'))
    # Expect columns: gvkey, tic, gs, formation_year
    if not {'gvkey','tic','gs','formation_year'}.issubset(gs.columns):
        raise ValueError("gs_table must have columns: ['gvkey','tic','gs','formation_year']")

    df = df.merge(gs[['gvkey','tic','formation_year','gs']],
                  on=['gvkey','tic','formation_year'], how='left')

    # Keep only necessary columns for decile formation
    chars = df[['formation_year','gvkey','tic','bm','ep','cp','gs','ME']].dropna(subset=['formation_year'])

    def one_char_deciles(chars_df: pd.DataFrame, char: str):
        # Use available firms with non-null characteristic
        cs = chars_df[['formation_year','gvkey','tic',char,'ME']].dropna(subset=[char]).copy()

        out_rows = []
        for Y, g in cs.groupby('formation_year', sort=True):
            # Need enough distinct values to form bins
            if g[char].nunique(dropna=True) < n_bins:
                continue

            g = g.copy()
            # Rank before qcut to avoid duplicate-edge errors
            rk = g[char].rank(method='first')
            try:
                g['decile'] = pd.qcut(rk, n_bins, labels=False) + 1  # 1..n_bins
            except ValueError:
                # still not enough spread
                continue

            members = (g.groupby('decile', sort=True)[id_col]
                         .apply(list)
                         .reindex(range(1, n_bins+1), fill_value=[])
                         .to_dict())
            out_rows.append((int(Y), members))

        # Build nested dict: deciles_by_year[year][decile] -> list
        deciles_by_year = {}
        for Y, members in out_rows:
            deciles_by_year[Y] = {int(k): v for k, v in members.items()}
        return deciles_by_year

    return {
        'bm': one_char_deciles(chars, 'bm'),
        'ep': one_char_deciles(chars, 'ep'),
        'cp': one_char_deciles(chars, 'cp'),
        'gs': one_char_deciles(chars, 'gs'),  # uses merged gs from gs_table
    }

# note that decile 1 is value for gs, but glamour for other ratios
deciles_by_year = build_decile_memberships_with_gs(df_statements, gs_table, id_col='tic', n_bins=10)

In [62]:
# example: show a couple stocks in the first decile in year 2020
# here, lower decile = low B/M ratio = glamour stock
deciles_by_year['bm'][2020][1][:5]

['AAL', 'WDDD', 'PBAJ', 'ANDR', 'BA']

In [63]:
def _as_period_m(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure monthly PeriodIndex; tolerate string or Timestamp index."""
    if isinstance(df.index, pd.PeriodIndex) and df.index.freqstr == "M":
        return df
    try:
        idx = pd.PeriodIndex(df.index, freq="M")
    except Exception:
        idx = pd.to_datetime(df.index).to_period("M")
    out = df.copy()
    out.index = idx
    return out

def _compound_12m(window: pd.Series) -> float:
    """Compound 12 monthly decimal returns; require all 12 finite values."""
    if window.shape[0] != 12:
        return np.nan
    w = window.astype(float)
    if not np.isfinite(w).all():
        return np.nan
    return float((1.0 + w).prod() - 1.0)

def _nanmean_rowwise(arr: np.ndarray, axis=0):
    """nanmean without warnings: NaN if all-NaN."""
    if arr.size == 0:
        return np.full((arr.shape[1-axis],), np.nan, dtype=float)
    counts = np.sum(~np.isnan(arr), axis=axis, dtype=float)
    sums = np.nansum(arr, axis=axis)
    out = np.divide(sums, counts, out=np.full_like(sums, np.nan, dtype=float), where=counts>0)
    return out

def _nanprod1p_over_row(row: np.ndarray) -> float:
    """Compound across (1+r) ignoring NaNs; NaN if none."""
    valid = row[np.isfinite(row)]
    if valid.size == 0:
        return np.nan
    return float(np.prod(1.0 + valid) - 1.0)

# --- core: post-formation using df_prices monthly RETURNS (decimal) ---

def postformation_returns_5y_from_df(df_prices: pd.DataFrame,
                                     ticker: str,
                                     formation_year: int) -> list[float]:
    """
    df_prices: monthly RETURNS (decimal), index monthly, cols tickers.
    R_k windows: May(Y+k-1)..Apr(Y+k).
    """
    if ticker not in df_prices.columns:
        return [np.nan]*5

    s = _as_period_m(df_prices)[ticker]

    out = []
    for k in range(5):
        y_start = formation_year + k
        p0 = pd.Period(f"{y_start}-05", freq="M")
        months = [p0 + i for i in range(12)]
        try:
            window = s.loc[months]
        except KeyError:
            out.append(np.nan)
            continue
        out.append(_compound_12m(window))
    return out

# --- cached wrapper bound to df_prices ---

def make_pf5y_cached(df_prices: pd.DataFrame):
    dfp = _as_period_m(df_prices)

    @lru_cache(maxsize=None)
    def _pf5y_cached(ticker: str, formation_year: int):
        try:
            vals = postformation_returns_5y_from_df(dfp, ticker, formation_year)
            if vals is None or len(vals) != 5:
                return (np.nan,)*5
            return tuple(float(x) if x is not None else np.nan for x in vals)
        except Exception:
            return (np.nan,)*5
    _pf5y_cached.cache_clear()

    return _pf5y_cached

# --- cohort & panel ---

def _cohort_equal_weight_R1toR5(tickers, formation_year, _pf5y_cached_fn):
    if not tickers:
        return [np.nan]*5
    rets = np.array([_pf5y_cached_fn(t, formation_year) for t in tickers], dtype=float)  # [N x 5]
    out = _nanmean_rowwise(rets, axis=0)  # -> [5], NaN if cohort has no valid data
    return out.tolist()

def build_panel_for_char(deciles_by_year, df_prices, char_key='bm', deciles=10, years=None):
    by_year = deciles_by_year[char_key]
    Ys_all = sorted(by_year.keys())
    years = Ys_all if years is None else [y for y in years if y in by_year]

    _pf5y_cached_fn = make_pf5y_cached(df_prices)

    sum_R = np.zeros((5, deciles), dtype=float)
    cnt_R = np.zeros((5, deciles), dtype=float)

    for Y in years:
        for d in range(1, deciles+1):
            tickers = by_year.get(Y, {}).get(d, [])
            r1to5 = np.array(_cohort_equal_weight_R1toR5(tickers, Y, _pf5y_cached_fn), dtype=float)
            mask = np.isfinite(r1to5)
            sum_R[mask, d-1] += r1to5[mask]
            cnt_R[mask, d-1] += 1.0

    with np.errstate(invalid='ignore', divide='ignore'):
        panel = sum_R / np.where(cnt_R==0.0, np.nan, cnt_R)

    panel_df = pd.DataFrame(panel,
                            index=[f"R{i}" for i in range(1,6)],
                            columns=[str(d) for d in range(1, deciles+1)])

    ar = pd.Series(_nanmean_rowwise(panel, axis=0), index=panel_df.columns, name="AR")
    cr5 = pd.Series([_nanprod1p_over_row(panel[:, j]) for j in range(panel.shape[1])],
                    index=panel_df.columns, name="CR_5y")

    return panel_df, ar, cr5

In [46]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='bm', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.230864  0.165150  0.154330  0.130206  0.152722  0.189832  0.141165   
R2  0.122684  0.109656  0.125627  0.149815  0.116547  0.110811  0.117326   
R3  0.120758  0.112367  0.149921  0.117748  0.121103  0.116944  0.125622   
R4  0.085786  0.111583  0.101683  0.110637  0.099287  0.108779  0.114245   
R5  0.119981  0.132974  0.113424  0.097341  0.110731  0.127159  0.110761   

           8         9        10  
R1  0.172715  0.202299  0.284169  
R2  0.132551  0.131110  0.214504  
R3  0.114867  0.147945  0.186675  
R4  0.133613  0.148052  0.163335  
R5  0.093090  0.127691  0.171839  
1     0.136015
2     0.126346
3     0.128997
4     0.121149
5     0.120078
6     0.130705
7     0.121824
8     0.129367
9     0.151419
10    0.204105
Name: AR, dtype: float64
1     0.883364
2     0.811259
3     0.832777
4     0.770284
5     0.761842
6     0.844962
7     0.776324
8     0.834823
9     1.021109
10    1.523051
Name: CR

In [47]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='cp', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.227251  0.173466  0.160419  0.157552  0.139625  0.162757  0.146482   
R2  0.178292  0.085334  0.064805  0.136292  0.126109  0.128571  0.117178   
R3  0.111257  0.115339  0.051022  0.144934  0.123267  0.110023  0.136120   
R4  0.074448  0.080854  0.097834  0.100324  0.093644  0.101602  0.118173   
R5  0.117135  0.163243  0.107877  0.119048  0.107860  0.112476  0.127549   

           8         9        10  
R1  0.173720  0.181839  0.291467  
R2  0.131451  0.151315  0.207648  
R3  0.135601  0.137397  0.208813  
R4  0.124371  0.134966  0.216348  
R5  0.114344  0.108298  0.132491  
1     0.141677
2     0.123647
3     0.096391
4     0.131630
5     0.118101
6     0.123086
7     0.129100
8     0.135897
9     0.142763
10    0.211353
Name: AR, dtype: float64
1     0.928822
2     0.785988
3     0.579520
4     0.854301
5     0.746572
6     0.785112
7     0.834671
8     0.889535
9     0.946724
10    1.597021
Name: CR

In [48]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='ep', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.218511  0.259526  0.219327  0.186359  0.162162  0.141284  0.138132   
R2  0.175050  0.149930  0.129714  0.152518  0.104449  0.108621  0.128862   
R3  0.178587  0.099141  0.124906  0.089504  0.137267  0.133705  0.134732   
R4  0.197867  0.096823  0.143069  0.111286  0.108970  0.099451  0.107490   
R5  0.146087  0.097596  0.164189  0.114651  0.134431  0.109840  0.107333   

           8         9        10  
R1  0.160866  0.151648  0.208449  
R2  0.135093  0.138354  0.144691  
R3  0.120605  0.125472  0.174058  
R4  0.102602  0.132027  0.136950  
R5  0.111579  0.105218  0.123138  
1     0.183220
2     0.140603
3     0.156241
4     0.130864
5     0.129456
6     0.118580
7     0.123310
8     0.126149
9     0.130544
10    0.157457
Name: AR, dtype: float64
1     1.316720
2     0.916511
3     1.062056
4     0.845260
5     0.836424
6     0.750302
7     0.787907
8     0.809778
9     0.846019
10    1.073866
Name: CR

In [ ]:
# Load the monthly returns you already built
df_prices = pd.read_csv("data/monthly_returns4.csv", index_col=0)

# Build panel for, say, book-to-market deciles
panel_df, ar, cr5 = build_panel_for_char(deciles_by_year, df_prices, char_key='gs', deciles=10)

print(panel_df)  # rows R1..R5 by decile
print(ar)        # average annual return across R1..R5 per decile
print(cr5)       # 5-year compounded return per decile


           1         2         3         4         5         6         7  \
R1  0.183009  0.146046  0.124904  0.160756  0.123126  0.143935  0.115408   
R2  0.123007  0.120306  0.144018  0.117842  0.101011  0.125919  0.124240   
R3  0.140216  0.139340  0.149814  0.139186  0.137195  0.169998  0.134152   
R4  0.172035  0.187767  0.131890  0.141335  0.130330  0.178253  0.156298   
R5  0.157961  0.126892  0.168525  0.119723  0.182910  0.119861  0.109558   

           8         9        10  
R1  0.116074  0.096647  0.082407  
R2  0.153553  0.119221  0.105054  
R3  0.141844  0.110344  0.111434  
R4  0.152605  0.236290  0.147169  
R5  0.129683  0.166986  0.127041  
1     0.155246
2     0.144070
3     0.143830
4     0.135768
5     0.134914
6     0.147593
7     0.127931
8     0.138752
9     0.145898
10    0.114621
Name: AR, dtype: float64
1     1.055854
2     0.957968
3     0.957125
4     0.889032
5     0.880234
6     0.988367
7     0.824667
8     0.914144
9     0.966195
10    0.718798
Name: CR

In [64]:
def build_3x3_grid(df_statements: pd.DataFrame,
                   char1: str = 'gs',      # e.g., past growth (GS)
                   char2: str = 'bm',      # e.g., value proxy (BM)
                   id_col: str = 'tic',
                   require_me_positive: bool = True,
                   min_names_per_year: int = 30):
    """
    Build LSV-style 3x3 portfolios by formation_year using 30/40/30 buckets on two characteristics.
    Returns: dict -> grid[year][(b1, b2)] = list[tickers], where b1,b2 ∈ {1(low),2(mid),3(high)}.

    Notes:
      - For GS: low(1)=value, high(3)=glamour
      - For BM/EP/CP: high(3)=value, low(1)=glamour
      - This function does NOT flip signs — it just bins the raw values. Interpret accordingly.
    """

    df = df_statements.copy()

    # Ensure formation_year exists; if not, derive from fyear+1
    if 'formation_year' not in df.columns and 'fyear' in df.columns:
        df['formation_year'] = pd.to_numeric(df['fyear'], errors='coerce').astype('Int64') + 1

    if 'formation_year' not in df.columns:
        raise ValueError("df_statements must have 'formation_year' (or 'fyear' to derive it).")

    # Optional filter: positive market equity to avoid junk
    if require_me_positive:
        if not {'prcc_f','csho'}.issubset(df.columns):
            raise ValueError("require_me_positive=True needs 'prcc_f' and 'csho' columns.")
        me = df['prcc_f'] * df['csho']
        df = df.loc[(me > 0)]

    # Keep only rows with both chars present
    need_cols = ['formation_year', id_col, char1, char2]
    for c in need_cols:
        if c not in df.columns:
            raise ValueError(f"Missing column '{c}' needed for sorting.")
    work = df[need_cols].dropna(subset=[char1, char2, 'formation_year']).copy()

    # Helper: map percentile rank to 3 buckets (30/40/30)
    def to_3bucket(pct: pd.Series) -> pd.Series:
        # pct is in (0,1]; make inclusive edges consistent with 30/40/30
        return pd.cut(pct,
                      bins=[0.0, 0.30, 0.70, 1.0],
                      labels=[1, 2, 3],
                      include_lowest=True,
                      right=True).astype('Int64')

    # Build per-year grids
    grid = {}
    for Y, g in work.groupby('formation_year'):
        g = g.copy()
        # skip tiny cross-sections
        if len(g) < min_names_per_year:
            continue

        # Percentile rank within the year for each characteristic.
        # rank(pct=True) returns (1..N)/N; use method='average' for tie robustness.
        g['pct1'] = g[char1].rank(method='average', pct=True)
        g['pct2'] = g[char2].rank(method='average', pct=True)

        # Map to 3 buckets: 1=low, 2=mid, 3=high
        g['b1'] = to_3bucket(g['pct1'])
        g['b2'] = to_3bucket(g['pct2'])

        # If either side collapses (e.g., not enough unique values), skip year
        if g['b1'].nunique(dropna=True) < 3 or g['b2'].nunique(dropna=True) < 3:
            continue

        # Collect members
        portfolios = {}
        for i in (1, 2, 3):
            for j in (1, 2, 3):
                names = g.loc[(g['b1'] == i) & (g['b2'] == j), id_col].dropna().astype(str).tolist()
                portfolios[(int(i), int(j))] = names

        grid[int(Y)] = portfolios

    return grid

In [67]:
def add_basic_characters(df_statements: pd.DataFrame) -> pd.DataFrame:
    """
    Ensures df_statements has basic characteristics:
    'bm', 'ep', 'cp', and 'ME' (market equity).
    """
    df = df_statements.copy()
    
    # Market Equity
    df['ME'] = df['prcc_f'] * df['csho']  # prcc_f: price, csho: shares outstanding
    
    # Replace invalid values
    df.loc[df['ME'] <= 0, 'ME'] = np.nan
    
    # Ratios
    df['bm'] = df['ceq']   / df['ME']    # book-to-market
    df['ep'] = df['ni']    / df['ME']    # earnings-to-price
    df['cp'] = df['oancf'] / df['ME']    # cash flow-to-price
    
    return df

def add_gs_to_statements(df_statements, gs_table):
    df = add_basic_characters(df_statements)  # ensure bm etc. are present
    df['formation_year'] = df['fyear'].astype(int) + 1
    merge_cols = ['gvkey', 'tic', 'formation_year']
    return df.merge(gs_table, on=merge_cols, how='left')


In [74]:
df_statements_augmented = add_gs_to_statements(df_statements, gs_table)

In [75]:
df_statements_augmented.columns

Index(['Unnamed: 0', 'gvkey', 'tic', 'fyear', 'datadate', 'conm', 'revt', 'ni',
       'emp', 'prcc_f', 'ceq', 'oancf', 'at', 'dltt', 'act', 'lct', 'csho',
       'cogs', 'ME', 'bm', 'ep', 'cp', 'formation_year', 'gs'],
      dtype='object')

In [ ]:
import numpy as np
import pandas as pd

# ---------- index normalization ----------

def _as_period_m(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return df with a monthly PeriodIndex (freq='M'), sorted.
    Works whether the original index is strings 'YYYY-MM', datetimes, or integers like 200601.
    """
    out = df.copy()

    # If already a monthly PeriodIndex, just sort and return
    if isinstance(out.index, pd.PeriodIndex) and (out.index.freqstr == "M" or getattr(out.index.freq, "name", None) == "M"):
        return out.sort_index()

    # Convert to datetime (coerce errors to NaT), then to monthly PeriodIndex
    out.index = pd.to_datetime(out.index, errors="coerce").to_period("M")
    return out.sort_index()

def _as_period_m_unique(df: pd.DataFrame, how: str = "last") -> pd.DataFrame:
    """
    Ensure a unique monthly PeriodIndex (freq='M').
    If duplicates exist for a month, collapse them via:
      - 'last' (default), 'first', or 'mean'.
    """
    out = _as_period_m(df)

    if out.index.has_duplicates:
        if how == "mean":
            out = out.groupby(level=0).mean(numeric_only=True)
        elif how == "first":
            out = out[~out.index.duplicated(keep="first")]
        else:  # "last"
            out = out[~out.index.duplicated(keep="last")]

    return out.sort_index()

# ---------- return compounding helpers ----------

def _compound_12m(window: pd.Series, min_months: int = 12, annualize: bool = False) -> float:
    """
    Compound monthly DECIMAL returns in `window` (length up to 12).
    If fewer than `min_months` finite values, return NaN.
    If `annualize` and partial (>=min_months but <12), geometrically annualize to 12 months.
    """
    w = pd.to_numeric(window, errors="coerce").astype(float).dropna()
    n = int(w.shape[0])
    if n < min_months:
        return np.nan
    prod = float((1.0 + w).prod() - 1.0)
    if annualize and n < 12:
        return float((1.0 + prod) ** (12.0 / n) - 1.0)
    return prod

def _nanprod1p(row: np.ndarray) -> float:
    """Compound across (1+r) ignoring NaNs; NaN if none."""
    valid = row[np.isfinite(row)]
    if valid.size == 0:
        return np.nan
    return float(np.prod(1.0 + valid) - 1.0)

def _months_window(formation_year: int, k: int = 0) -> list[pd.Period]:
    """May(Y+k) .. Apr(Y+k+1) as monthly Periods."""
    y0 = formation_year + k
    start = pd.Period(f"{y0}-05", freq="M")
    return [start + i for i in range(12)]

def _has_min_coverage(s: pd.Series, formation_year: int, min_months: int) -> bool:
    """Return True if at least `min_months` months exist in the R1 window May..Apr."""
    # Source index must be unique monthly PeriodIndex already; still guard for duplicates
    if s.index.has_duplicates:
        s = s[~s.index.duplicated(keep="last")].sort_index()
    win = s.reindex(_months_window(formation_year, 0))
    return int(pd.to_numeric(win, errors="coerce").notna().sum()) >= min_months

# ---------- per-ticker / per-cell returns ----------

def _ticker_r1to5(df_prices: pd.DataFrame, ticker: str, formation_year: int,
                  min_months: int = 12, annualize_partial: bool = False) -> list[float]:
    """
    R_k windows: May(Y+k-1)..Apr(Y+k) using monthly decimal returns.
    Returns [R1..R5]; NaN if a window is incomplete (< min_months).
    """
    if ticker not in df_prices.columns:
        return [np.nan]*5

    s = pd.to_numeric(df_prices[ticker], errors="coerce")

    # Belt-and-suspenders: ensure unique
    if s.index.has_duplicates:
        s = s[~s.index.duplicated(keep="last")].sort_index()

    out = []
    for k in range(5):
        months = _months_window(formation_year, k)
        win = s.reindex(months)
        out.append(_compound_12m(win, min_months=min_months, annualize=annualize_partial))
    return out

def _cell_r1to5_equal_weight(df_prices: pd.DataFrame, tickers: list[str], formation_year: int,
                             min_months: int = 12, annualize_partial: bool = False) -> list[float]:
    """
    Equal-weight across names; for each k, ignore NaNs from names that lack coverage.
    """
    if not tickers:
        return [np.nan]*5
    arr = np.array(
        [_ticker_r1to5(df_prices, t, formation_year, min_months=min_months, annualize_partial=annualize_partial)
         for t in tickers],
        dtype=float
    )  # [N,5]

    with np.errstate(all='ignore'):
        counts = np.sum(np.isfinite(arr), axis=0)
        sums   = np.nansum(arr, axis=0)
        out    = np.divide(sums, counts, out=np.full(5, np.nan, dtype=float), where=counts > 0)
    return out.tolist()

# ---------- main: compute post-formation returns for 3x3 grid ----------

def postformation_3x3(
    df_prices: pd.DataFrame,                              # monthly decimal returns
    grid_3x3: dict[int, dict[tuple, list[str]]],
    years: list[int] | None = None,
    min_months: int = 10,                                 # 12 for strict; 10 tolerates a couple missing months
    annualize_partial: bool = False,                      # set True if using min_months < 12 and you want annualization
    drop_cells_with_no_survivors: bool = False,
):
    """
    Returns:
      per_year: dict[Y][(i,j)] -> [R1..R5]
      panel_avg: DataFrame (rows R1..R5, cols MultiIndex (b1,b2)), averaged across years
      ar_3x3: 3x3 DataFrame of AR = mean(R1..R5)
      cr5_3x3: 3x3 DataFrame of CR(5y) = prod_t (1+Rt) - 1
    """
    # 1) Normalize index & remove duplicate months
    dfp = _as_period_m_unique(df_prices, how="last").copy()

    Ys_all = sorted(grid_3x3.keys())
    Ys = Ys_all if years is None else [y for y in years if y in grid_3x3]

    per_year: dict[int, dict[tuple, list[float]]] = {}
    cell_acc = {(i,j): [] for i in (1,2,3) for j in (1,2,3)}

    # 2) For each year/cell, keep only tickers that exist and have coverage in the first window (R1)
    for Y in Ys:
        cell_map = grid_3x3.get(Y, {})
        per_year[Y] = {}
        for i in (1,2,3):
            for j in (1,2,3):
                raw_names = cell_map.get((i,j), [])
                survivors = [t for t in raw_names
                             if (t in dfp.columns) and _has_min_coverage(dfp[t], Y, min_months=min_months)]
                if drop_cells_with_no_survivors and not survivors:
                    per_year[Y][(i,j)] = [np.nan]*5
                    cell_acc[(i,j)].append([np.nan]*5)
                    continue
                
                r1to5 = _cell_r1to5_equal_weight(
                    dfp, survivors, Y, min_months=min_months, annualize_partial=annualize_partial
                )
                per_year[Y][(i,j)] = r1to5
                cell_acc[(i,j)].append(r1to5)

    # 3) Average across years per cell
    cols = pd.MultiIndex.from_product([[1,2,3],[1,2,3]], names=['b1','b2'])
    panel_avg = pd.DataFrame(index=[f"R{k}" for k in range(1,6)], columns=cols, dtype=float)

    for (i,j), lst in cell_acc.items():
        if not lst:
            panel_avg[(i,j)] = np.nan
            continue
        arr = np.array(lst, dtype=float)  # [num_years, 5]
        with np.errstate(all='ignore'):
            counts = np.sum(np.isfinite(arr), axis=0)
            sums   = np.nansum(arr, axis=0)
            avg    = np.divide(sums, counts, out=np.full(5, np.nan, dtype=float), where=counts > 0)
        panel_avg[(i,j)] = avg

    # 4) AR and CR(5y) 3x3 tables
    ar_3x3  = pd.DataFrame(index=[1,2,3], columns=[1,2,3], dtype=float)
    cr5_3x3 = pd.DataFrame(index=[1,2,3], columns=[1,2,3], dtype=float)

    for i in (1,2,3):
        for j in (1,2,3):
            col = panel_avg[(i,j)].to_numpy()  # length 5
            if np.isfinite(col).any():
                ar_3x3.loc[i,j]  = float(np.nanmean(col))
                cr5_3x3.loc[i,j] = _nanprod1p(col)
            else:
                ar_3x3.loc[i,j] = np.nan
                cr5_3x3.loc[i,j] = np.nan

    ar_3x3.index.name = 'GS bucket (b1: 1=low,3=high)'
    ar_3x3.columns.name = 'BM bucket (b2: 1=low,3=high)'
    cr5_3x3.index.name = ar_3x3.index.name
    cr5_3x3.columns.name = ar_3x3.columns.name

    return per_year, panel_avg, ar_3x3, cr5_3x3


In [ ]:
grid_3x3 = build_3x3_grid(df_statements_augmented, char1='gs', char2='bm', id_col='tic')
per_year, panel_avg, ar_3x3, cr5_3x3 = postformation_3x3(df_prices, grid_3x3)

print("Panel average (R1..R5 by cell):")
print(panel_avg)

print("\nAR (mean of R1..R5) 3x3:")
print(ar_3x3)

dfp               AIR       AAL       ASA       PNW       PRG       ABT      WDDD  \
2003-01       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2003-02 -0.065126       NaN -0.094070 -0.006187 -0.085308 -0.060030  0.000000   
2003-03 -0.150562       NaN -0.037873  0.088408  0.044042  0.055868  0.000000   
2003-04  0.026455       NaN  0.015850 -0.000601  0.049628  0.080297  0.000000   
2003-05  0.159794       NaN  0.018440  0.154016  0.026004  0.103219  0.000000   
...           ...       ...       ...       ...       ...       ...       ...   
2025-03 -0.138880 -0.264808  0.234740  0.039988 -0.062390 -0.038838 -0.170000   
2025-04 -0.045187 -0.056872 -0.005309 -0.000735 -0.004235 -0.014323 -0.120482   
2025-05  0.148709  0.146734  0.055704 -0.041500  0.093703  0.026377 -0.178082   
2025-06  0.120176 -0.016652 -0.001488 -0.009897  0.022441  0.018191  0.416667   
2025-07  0.086059  0.024064 -0.019639  0.012853  0.084838 -0.072201 -0.964706   

            ACMTA      